In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import xgboost as xgb

from datetime import datetime

from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv("data/ech_apprentissage.csv",
                    sep = ';',
                    index_col = 'id',
                    low_memory = False)

X_test = pd.read_csv('data/ech_test.csv',
                     sep = ';',
                     index_col = 'id')

X = train.drop('prime_tot_ttc', axis = 1)
y = train.prime_tot_ttc

In [3]:
cat_variables = ['marque', 
                'codepostal',
                'energie_veh',
                 'profession',
                 'var6',
                 'var7',
                 'var8',
                 'var14',
                 'var16'
                ]

In [4]:
# X.fillna(0, inplace = True)
# X_test.fillna(0, inplace = True)

In [9]:
X.fillna(X.mean(axis=1, skipna=True), inplace = True)
X_test.fillna(X_test.mean(axis=1, skipna=True), inplace = True)

,annee_naissance,annee_permis,marque,puis_fiscale,anc_veh,codepostal,energie_veh,kmage_annuel,crm,profession,...,var13,var14,var15,var16,var17,var18,var19,var20,var21,var22
id,,,,,,,,,,,,,,,,,,,,,
300001,1992.0,2011.0,54,4,2,2,2,2983,88,9,...,3,0,0,1,0,20,2,0,1,0
300002,1991.0,2010.0,54,6,36,2,2,6496,50,9,...,5,0,0,4,1,20,1,0,1,0
300003,1976.0,1995.0,54,10,3,2,2,6971,50,9,...,5,0,7,1,1,21,1,0,1,0
300004,1976.0,1995.0,7,13,1,2,2,7293,68,9,...,4,0,0,1,1,22,1,0,1,0
300005,1984.0,2004.0,50,9,0,2,2,6982,50,9,...,5,0,2,4,1,16,0,0,1,1
300006,1990.0,2009.0,54,7,33,2,2,7000,100,12,...,5,0,0,4,1,16,0,0,1,1
300007,1980.0,1999.0,50,7,33,2,2,11451,100,12,...,5,0,0,4,1,17,0,1,1,1
300008,1965.0,1984.0,54,5,0,2,2,2906,54,8,...,3,0,0,3,1,18,0,1,1,1
300009,1971.0,1990.0,45,6,2,2,2,6633,125,8,...,4,0,0,1,1,20,0,0,1,1


In [11]:
encods = [LabelEncoder() for col in cat_variables]
for i, col in enumerate(cat_variables):
    X[col] = encods[i].fit_transform(X[col].astype(str))

encods_test = [LabelEncoder() for col in cat_variables]
for i, col in enumerate(cat_variables):
    X_test[col] = encods_test[i].fit_transform(X_test[col].astype(str))

In [12]:
# человеческая метрика
def mape2(preds, dtrain):
    y_true = dtrain.get_label()
    return 'mape2', np.mean(np.abs((y_true - preds) / y_true))

dtrain = xgb.DMatrix(X, y, missing=0)
dtest = xgb.DMatrix(X_test)

XGBoostError: b'There are NAN in the matrix, however, you did not set missing=NAN'

In [13]:
param = { }
num_round = 100
bst = xgb.train(param, dtrain, num_round)

NameError: name 'dtrain' is not defined

In [14]:
pred = bst.predict(dtest)

NameError: name 'bst' is not defined

In [ ]:
filename = 'xgb_' + datetime.now().strftime(format = '%d%m_%H%M') + '.csv'
pd.DataFrame({'id': X_test.index, 'pred': pred}).to_csv(filename,
                                                        index=False,
                                                        sep=';')